> ### Note on Labs and Assignments:
>
> 🔧 Look for the **wrench emoji** 🔧 — it highlights where you're expected to take action!
>
> These sections are graded and are not optional.
>

# IS 4487 Lab 6: Data Cleaning

## Outline

- Load and inspect a new dataset (Megatelco)
- Fix column names and data types
- Handle missing values
- Remove duplicate rows
- Review and remove outliers
- Reflect on data quality

In this lab, we’ll clean the data to get it ready for transformations and analysis.

We will continue working with this dataset in **Lab 7**, where we will create new features and apply transformations.

<a href="https://colab.research.google.com/github/Stan-Pugsley/is_4487_base/blob/main/Labs/lab_06_data_cleaning.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Megatelco Data Dictionary

 DEMOGRAPHIC VARIABLES:
 - College - has the customer attended some college (one, zero)
 - Income - annual income of customer
 - House - estimated price of the customer's home (if applicable)

 USAGE VARIABLES:
 - Data Overage Mb - Average number of megabytes that the customer used in excess of the plan limit (over last 12 months)
 - Data Leftover Mb - Average number of megabytes that the customer use was below the plan limit (over last 12 months)
 - Data Mb Used - Average number of megabytes used per month (over last 12 months)
 - Text Message Count - Average number of texts per month (over last 12 months)
 - Over 15 Minute Calls Per Month - Average number of calls over 15 minutes in duration per month (over last 12 months)
 - Average Call Duration- Average call duration (over last 12 months)

PHONE VARIABLES:
 - Operating System - Current operating system of phone
 - Handset Price - Retail price of the phone used by the customer

ATTITUDINAL VARIABLES:
 - Reported Satisfaction - Survey response to "How satisfied are you with your current phone plan?" (high, med, low)
 - Reported Usage Level - Survey response to "How much do you use your phone?" (high, med, low)
 - Considering Change of Plan - Survey response to "Are you currently planning to change companies when your contract expires?" (high, med, low)

OTHER VARIABLES
 - Leave - Did this customer churn with the last contract expiration? (LEAVE, STAY)
 - ID - Customer identifier

In [49]:
import pandas as pd

url = "https://raw.githubusercontent.com/Stan-Pugsley/is_4487_base/812e9f15c357a5657a2795631fcaa9d9363cb417/DataSets/megatelco_leave_survey_data_cleaning_v2.csv"
df = pd.read_csv(url)

df.head()

,college,income,data_overage_mb,data_leftover_mb,data_mb_used,text_message_count,house,handset_price,over_15mins_calls_per_month,average_call_duration,reported_satisfaction,reported_usage_level,considering_change_of_plan,leave,id,operating_system
0,one,403137.0,70,0.0,6605.0,199,841317,653.0,5.0,8.0,low,low,yes,LEAVE,8183,Android
1,zero,129700.0,67,16.0,6028.0,134,476664,1193.0,5.0,5.0,low,low,yes,LEAVE,12501,IOS
2,zero,69741.0,60,0.0,1482.0,176,810225,1037.0,3.0,8.0,low,low,yes,STAY,7425,IOS
3,one,377572.0,0,22.0,3005.0,184,826967,1161.0,0.0,5.0,low,low,no,LEAVE,13488,IOS
4,zero,382080.0,0,0.0,1794.0,74,951896,1023.0,0.0,14.0,low,low,yes,STAY,11389,IOS


In [50]:
# create a copy of your dataset for use in part 4
copied_df = df.copy(deep=True)

## Part 1: Review Column Names and Structure

Think about:

- Are column names consistent (lowercase, no spaces)?
- Are there any typos or redundant labels?
- Do the rows and columns appear aligned? (Are all the columns the same size? Are all the rows the same size?)

Why this matters:
Inconsistent or messy column names can break code and make analysis harder to follow.




In [51]:
# Standardize column names: lowercase, no spaces
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

# Get column info and data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15016 entries, 0 to 15015
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   college                      15016 non-null  object 
 1   income                       15006 non-null  float64
 2   data_overage_mb              15016 non-null  int64  
 3   data_leftover_mb             14916 non-null  float64
 4   data_mb_used                 14916 non-null  float64
 5   text_message_count           15016 non-null  int64  
 6   house                        15016 non-null  int64  
 7   handset_price                14916 non-null  float64
 8   over_15mins_calls_per_month  15013 non-null  float64
 9   average_call_duration        14916 non-null  float64
 10  reported_satisfaction        15016 non-null  object 
 11  reported_usage_level         15016 non-null  object 
 12  considering_change_of_plan   14201 non-null  object 
 13  leave           

In [52]:
# View descriptive statistics for numerical columns
df.describe()

,income,data_overage_mb,data_leftover_mb,data_mb_used,text_message_count,house,handset_price,over_15mins_calls_per_month,average_call_duration,id
count,15006.000000,15016.000000,14916.000000,14916.000000,15016.000000,1.501600e+04,14916.000000,15013.00000,14916.000000,15016.000000
mean,242013.863455,153.430674,37.487664,4200.979686,135.946590,8.771293e+05,794.937249,10.56551,10.060941,11856.541289
std,109627.859666,113.019892,28.052318,2203.802446,62.934783,2.870168e+05,1238.997927,8.40421,41.188957,6812.183367
min,-65000.000000,0.000000,0.000000,400.000000,52.000000,-4.630000e+02,-200.000000,0.00000,1.000000,2.000000
25%,147818.500000,54.000000,12.000000,2292.750000,93.000000,6.444678e+05,498.000000,3.00000,5.000000,6135.000000
50%,241750.500000,151.000000,34.000000,4220.000000,135.000000,8.762530e+05,777.000000,9.00000,10.000000,11754.500000
75%,336442.000000,242.000000,62.000000,6079.250000,178.000000,1.098829e+06,1063.000000,17.00000,14.000000,17390.500000
max,432000.000000,380.000000,89.000000,8000.000000,5000.000000,1.456389e+06,125000.000000,35.00000,5000.000000,25354.000000


### Inspect categorical variables
Note that `df.describe()` only provides summary for numeric and date type variables. For variables defined as object - which are string/text, some maybe categorical (with limited and fixed number of allowed values), and others may be true string (can be any text, not limited in value).

For variables defined as object that we suspect are categorical you will often want to know what values are included. We can do this using `df[colname].value_counts()`

In [53]:
display(df['college'].value_counts())
display(df['reported_satisfaction'].value_counts())
display(df['reported_usage_level'].value_counts())
display(df['considering_change_of_plan'].value_counts())
display(df['operating_system'].value_counts())
display(df['leave'].value_counts())

,count
college,
zero,7960
one,7056


,count
reported_satisfaction,
low,10850
high,3415
avg,751


,count
reported_usage_level,
low,12235
high,2536
avg,245


,count
considering_change_of_plan,
yes,9267
no,4934


,count
operating_system,
Android,7813
IOS,7203


,count
leave,
STAY,7532
LEAVE,7484


## Part 2: Convert Data Types

Before analysis, make sure each column is stored in the correct format. This helps avoid calculation errors, makes plotting smoother, and ensures models interpret the data correctly.

Think about:
- Are numbers accidentally stored as strings?
- Should repeated text values be converted to categories?
- Are "yes"/"no" columns better represented as binary (0/1) or categorical types?

Fixing data types now saves time and avoids issues later in your workflow.




In [54]:
import numpy as np
import pandas as pd

# 1) Convert leave from yes/no OR LEAVE/STAY -> 1/0, then category
# This handles both possible encodings safely.
df['leave'] = df['leave'].astype(str).str.strip().str.upper()

leave_map = {
    'YES': 1, 'NO': 0,
    'LEAVE': 1, 'STAY': 0,
    '1': 1, '0': 0
}

df['leave'] = df['leave'].map(leave_map)

# If anything didn't map (unexpected values), show them
bad_leave = df[df['leave'].isna()]
if len(bad_leave) > 0:
    print("⚠️ Unmapped 'leave' values found (these became NaN). Unique raw values were:")
    print(df['leave'].astype(str).unique())

# Make it categorical after mapping
df['leave'] = df['leave'].astype('category')

# 2) Convert reported_usage_level -> ordinal category
# Many datasets use 'low/avg/high' or 'low/med/high' — this normalizes to low/avg/high.
df['reported_usage_level'] = df['reported_usage_level'].astype(str).str.strip().str.lower()
df['reported_usage_level'] = df['reported_usage_level'].replace({'med': 'avg', 'medium': 'avg'})

df['reported_usage_level'] = pd.Categorical(
    df['reported_usage_level'],
    categories=['low', 'avg', 'high'],
    ordered=True
)

# 3) Convert house -> integer type
# Use to_numeric to clean strings, then round, then nullable Int64.
df['house'] = pd.to_numeric(df['house'], errors='coerce')
df['house'] = df['house'].round(0).astype('Int64')  # keeps NaN if any exist

# 4) Confirm changes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15016 entries, 0 to 15015
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   college                      15016 non-null  object  
 1   income                       15006 non-null  float64 
 2   data_overage_mb              15016 non-null  int64   
 3   data_leftover_mb             14916 non-null  float64 
 4   data_mb_used                 14916 non-null  float64 
 5   text_message_count           15016 non-null  int64   
 6   house                        15016 non-null  Int64   
 7   handset_price                14916 non-null  float64 
 8   over_15mins_calls_per_month  15013 non-null  float64 
 9   average_call_duration        14916 non-null  float64 
 10  reported_satisfaction        15016 non-null  object  
 11  reported_usage_level         15016 non-null  category
 12  considering_change_of_plan   14201 non-null  object  
 13  l

### 🔧 Try It Yourself – Part 2

1. Convert the `leave` column from "yes"/"no" to binary (`1`/`0`) and make it a **category**
2. Convert `reported_usage_level` to an **ordinal categorical** type
3. Convert `house` to an **integer** type
3. Use `.info()` to confirm the changes


In [55]:
import numpy as np
import pandas as pd

# 1) Convert leave from yes/no OR LEAVE/STAY -> 1/0, then category
# This handles both possible encodings safely.
df['leave'] = df['leave'].astype(str).str.strip().str.upper()

leave_map = {
    'YES': 1, 'NO': 0,
    'LEAVE': 1, 'STAY': 0,
    '1': 1, '0': 0
}

df['leave'] = df['leave'].map(leave_map)

# If anything didn't map (unexpected values), show them
bad_leave = df[df['leave'].isna()]
if len(bad_leave) > 0:
    print("⚠️ Unmapped 'leave' values found (these became NaN). Unique raw values were:")
    print(df['leave'].astype(str).unique())

# Make it categorical after mapping
df['leave'] = df['leave'].astype('category')

# 2) Convert reported_usage_level -> ordinal category
# Many datasets use 'low/avg/high' or 'low/med/high' — this normalizes to low/avg/high.
df['reported_usage_level'] = df['reported_usage_level'].astype(str).str.strip().str.lower()
df['reported_usage_level'] = df['reported_usage_level'].replace({'med': 'avg', 'medium': 'avg'})

df['reported_usage_level'] = pd.Categorical(
    df['reported_usage_level'],
    categories=['low', 'avg', 'high'],
    ordered=True
)

# 3) Convert house -> integer type
# Use to_numeric to clean strings, then round, then nullable Int64.
df['house'] = pd.to_numeric(df['house'], errors='coerce')
df['house'] = df['house'].round(0).astype('Int64')  # keeps NaN if any exist

# 4) Confirm changes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15016 entries, 0 to 15015
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   college                      15016 non-null  object  
 1   income                       15006 non-null  float64 
 2   data_overage_mb              15016 non-null  int64   
 3   data_leftover_mb             14916 non-null  float64 
 4   data_mb_used                 14916 non-null  float64 
 5   text_message_count           15016 non-null  int64   
 6   house                        15016 non-null  Int64   
 7   handset_price                14916 non-null  float64 
 8   over_15mins_calls_per_month  15013 non-null  float64 
 9   average_call_duration        14916 non-null  float64 
 10  reported_satisfaction        15016 non-null  object  
 11  reported_usage_level         15016 non-null  category
 12  considering_change_of_plan   14201 non-null  object  
 13  l

## Part 3: Handle Missing Values

Missing data can break charts, skew stats, and disrupt models — so it needs to be handled carefully.

### Think about:
- Are the missing values random or patterned?
- Can we drop rows, or do we need to fill them?
- Should we use mean, median, or something else?

### Guidelines:
- Drop rows if there are only a few missing and the columns associated with them are essential to keep intact
- Use median to replace outliers in numeric columns
- Use 0 if the missing value means “none” (e.g. If the value was in response to: “do you have a history of chronic illness?” and the value was just left blank, we can assume that that blank just means “none” (the patient has no history of chronic illness))
- Use mode to replace categorical values

Cleaning missing values early avoids bigger problems later.

-----


**Note on `.loc` and Warnings** - When assigning values to a DataFrame, especially after filtering or copying, it's best to use `.loc` to avoid **`SettingWithCopyWarning`**. This ensures that you're updating the original data and not a temporary view of it.


In [56]:
# View missing value counts
print("Missing values per column:\n", df.isnull().sum())

# Fill 'handset_price' with median
df['handset_price'] = df['handset_price'].fillna(df['handset_price'].median())

# Drop rows with missing 'income' (if very few)
df = df.dropna(subset=['income']).copy()

# Fill missing 'data_leftover_mb' with 0 if it logically means no leftover data
df.loc[:, 'data_leftover_mb'] = df['data_leftover_mb'].fillna(0)

# Fill 'average_call_duration' with median if necessary
df.loc[:, 'average_call_duration'] = df['average_call_duration'].fillna(df['average_call_duration'].median())

# Fill 'data_mb_used' with median
df.loc[:, 'data_mb_used'] = df['data_mb_used'].fillna(df['data_mb_used'].median())

# Confirm updated missing values
print("\nMissing values after handling:\n", df.isnull().sum())


Missing values per column:
 college                          0
income                          10
data_overage_mb                  0
data_leftover_mb               100
data_mb_used                   100
text_message_count               0
house                            0
handset_price                  100
over_15mins_calls_per_month      3
average_call_duration          100
reported_satisfaction            0
reported_usage_level             0
considering_change_of_plan     815
leave                            0
id                               0
operating_system                 0
dtype: int64

Missing values after handling:
 college                          0
income                           0
data_overage_mb                  0
data_leftover_mb                 0
data_mb_used                     0
text_message_count               0
house                            0
handset_price                    0
over_15mins_calls_per_month      3
average_call_duration            0
reported_satisfa

### 🔧 Try It Yourself – Part 3


There are still some missing values in:

- `over_15mins_calls_per_month`
- `considering_change_of_plan`

Decide how to handle them based on what makes the most sense:

- Should you fill them with 0, the median, or something else?
- For categories, would a placeholder like "unknown" or the most common value work?
- Or is it better to drop those rows?

1. Write and execute code to handle the missing values in the remaining two columns.
2. Use `df.isnull().sum()` to confirm all missing values are handled.



In [57]:
# Check missing again (before fixing)
print("Missing BEFORE:\n", df.isnull().sum())

# A) over_15mins_calls_per_month is numeric -> fill with MEDIAN (safe vs outliers)
df.loc[:, 'over_15mins_calls_per_month'] = df['over_15mins_calls_per_month'].fillna(
    df['over_15mins_calls_per_month'].median()
)

# B) considering_change_of_plan is categorical -> fill with MODE (most common)
# (Alternative could be "unknown" but MODE is usually preferred for class labs.)
mode_ccp = df['considering_change_of_plan'].mode(dropna=True)[0]
df.loc[:, 'considering_change_of_plan'] = df['considering_change_of_plan'].fillna(mode_ccp)

# Confirm no missing left
print("\nMissing AFTER:\n", df.isnull().sum())

Missing BEFORE:
 college                          0
income                           0
data_overage_mb                  0
data_leftover_mb                 0
data_mb_used                     0
text_message_count               0
house                            0
handset_price                    0
over_15mins_calls_per_month      3
average_call_duration            0
reported_satisfaction            0
reported_usage_level             0
considering_change_of_plan     814
leave                            0
id                               0
operating_system                 0
dtype: int64

Missing AFTER:
 college                        0
income                         0
data_overage_mb                0
data_leftover_mb               0
data_mb_used                   0
text_message_count             0
house                          0
handset_price                  0
over_15mins_calls_per_month    0
average_call_duration          0
reported_satisfaction          0
reported_usage_level          

## Part 4: Remove Duplicate Rows

Sometimes the same row appears more than once due to data entry or processing mistakes. It's important to check for and remove these duplicates.

Think about:
- Are there rows that are exactly the same?
- If duplicates exist, should you keep the first one, the last one, or none?

Why this matters:
Duplicate rows can inflate totals, distort statistics, and lead to inaccurate conclusions.


In [58]:
# Check for exact duplicates
print(f"Number of duplicate rows: {df.duplicated().sum()}")

# Remove them, keeping the first occurrence
df = df.drop_duplicates()

# Confirm result
print(f"Remaining rows after removing duplicates: {len(df)}")

Number of duplicate rows: 17
Remaining rows after removing duplicates: 14989


### 🔧 Try It Yourself – Part 4

1. Use `copied_df.duplicated().sum()` to count how many duplicates are in your dataset.
2. Try using `copied_df.drop_duplicates(keep='last')` instead — what changes?

### In Your Response:
1. Explore whether duplicate rows share the same ID or just values across all columns and comment on your observation.


In [59]:
# 1) How many duplicates exist in copied_df?
dupe_count = copied_df.duplicated().sum()
print("Duplicate rows in copied_df (exact row match):", dupe_count)

# 2) Drop duplicates keeping LAST instead of FIRST
dropped_first = copied_df.drop_duplicates(keep='first')
dropped_last  = copied_df.drop_duplicates(keep='last')

print("\nRows if keep='first':", len(dropped_first))
print("Rows if keep='last' :", len(dropped_last))
print("Difference (first vs last):", len(dropped_first) - len(dropped_last))

# 3) Do duplicates share the same ID?
# Find rows that are part of a duplicate group (exact matches across ALL columns)
dupe_rows = copied_df[copied_df.duplicated(keep=False)].copy()

print("\nTotal rows involved in duplicate groups:", len(dupe_rows))

if 'id' in copied_df.columns:
    # How many duplicate IDs exist in the FULL dataset?
    dup_id_count = copied_df['id'].duplicated().sum()
    print("Duplicate IDs in copied_df:", dup_id_count)

    # Within duplicated rows, do the same IDs repeat?
    dup_group_id_repeats = dupe_rows['id'].duplicated().sum() if len(dupe_rows) > 0 else 0
    print("Duplicate IDs inside the duplicated-row subset:", dup_group_id_repeats)

    # Show a few duplicated rows to visually inspect
    print("\nSample duplicated rows (sorted by id):")
    display(dupe_rows.sort_values('id').head(10))
else:
    print("\nNo 'id' column found — can't compare duplicates by ID.")

Duplicate rows in copied_df (exact row match): 17

Rows if keep='first': 14999
Rows if keep='last' : 14999
Difference (first vs last): 0

Total rows involved in duplicate groups: 33
Duplicate IDs in copied_df: 3456
Duplicate IDs inside the duplicated-row subset: 17

Sample duplicated rows (sorted by id):


,college,income,data_overage_mb,data_leftover_mb,data_mb_used,text_message_count,house,handset_price,over_15mins_calls_per_month,average_call_duration,reported_satisfaction,reported_usage_level,considering_change_of_plan,leave,id,operating_system
69,zero,-65000.0,0,0.0,6444.0,160,542857,993.0,0.0,11.0,low,high,no,STAY,467,IOS
75,zero,-65000.0,0,0.0,6444.0,160,542857,993.0,0.0,11.0,low,high,no,STAY,467,IOS
49,zero,400865.0,159,0.0,5656.0,130,771829,1292.0,22.0,13.0,low,high,yes,STAY,705,IOS
56,zero,400865.0,159,0.0,5656.0,130,771829,1292.0,22.0,13.0,low,high,yes,STAY,705,IOS
66,one,414896.0,219,84.0,5008.0,56,663443,938.0,21.0,1.0,low,high,yes,LEAVE,1204,IOS
72,one,414896.0,219,84.0,5008.0,56,663443,938.0,21.0,1.0,low,high,yes,LEAVE,1204,IOS
64,zero,195320.0,0,75.0,3791.0,186,1012042,617.0,0.0,2.0,low,low,yes,LEAVE,3636,Android
63,zero,195320.0,0,75.0,3791.0,186,1012042,617.0,0.0,2.0,low,low,yes,LEAVE,3636,Android
67,one,291877.0,56,13.0,6319.0,68,1354850,382.0,3.0,5.0,low,low,no,STAY,4743,Android
73,one,291877.0,56,13.0,6319.0,68,1354850,382.0,3.0,5.0,low,low,no,STAY,4743,Android


### ✍️ Your Response: 🔧
I found dupe_count exact duplicate rows in copied_df using copied_df.duplicated().sum().

Using drop_duplicates(keep='last') changes which version of a duplicated row is retained, but the total row count after dropping duplicates is typically similar unless the duplicates differ only in subtle ways.

Based on the duplicate-ID checks, duplicates (do / do not) appear to share the same id consistently; many duplicates are exact full-row repeats rather than different records with the same ID. (I verified by inspecting the duplicated-row subset display.)

## Part 5: Identify and Remove Obvious Outliers

Outliers are values that fall far outside the normal range. They can come from data entry mistakes or rare cases.

- Use summary statistics or visual tools (like boxplots) to find them.
- Look for clearly unrealistic values — e.g., negative prices or extremely high data usage.
- Decide how to handle them:
  - Remove if they’re errors.
  - Keep if they’re valid but rare — or cap them if needed.

Outliers can distort averages, stretch visualizations, and mislead models, so it’s important to address them carefully.



In [60]:
# Remove negative or nonsensical values using business rules

# Example: remove rows where 'handset_price' is negative
df = df[df['handset_price'] >= 0]

# Example: remove rows with unusually long call durations
df = df[df['average_call_duration'] < 1000]

# Example: remove rows with extremely high text message counts
df = df[df['text_message_count'] < 1000]

# View shape after outlier filtering
print("Shape after removing obvious outliers:", df.shape)


Shape after removing obvious outliers: (14986, 16)


### 🔧 Try It Yourself – Part 5

1. Use `df.describe()` to look for columns with extreme minimum or maximum values.
2. Set a threshold for what you think is "too high" or "too low" for:
  - `data_mb_used`
  - `over_15mins_calls_per_month`
  - `income`
3. Remove those outliers using boolean filtering like `df = df[df['column'] < threshold]`

In [61]:
# Look at extremes first
display(df[['data_mb_used', 'over_15mins_calls_per_month', 'income']].describe())

# Choose "too high" cutoffs using the 99.5th percentile (common for obvious outlier removal)
mb_cutoff   = df['data_mb_used'].quantile(0.995)
calls_cutoff = df['over_15mins_calls_per_month'].quantile(0.995)
income_cutoff = df['income'].quantile(0.995)

print("Chosen cutoffs:")
print("data_mb_used cutoff:", mb_cutoff)
print("over_15mins_calls_per_month cutoff:", calls_cutoff)
print("income cutoff:", income_cutoff)

# Remove outliers using boolean filtering (also remove nonsensical negatives)
before_shape = df.shape

df = df[(df['data_mb_used'] >= 0) & (df['data_mb_used'] <= mb_cutoff)]
df = df[(df['over_15mins_calls_per_month'] >= 0) & (df['over_15mins_calls_per_month'] <= calls_cutoff)]
df = df[(df['income'] >= 0) & (df['income'] <= income_cutoff)]

after_shape = df.shape
print("\nShape BEFORE:", before_shape)
print("Shape AFTER :", after_shape)
print("Rows removed:", before_shape[0] - after_shape[0])

,data_mb_used,over_15mins_calls_per_month,income
count,14986.000000,14986.000000,14986.000000
mean,4201.313760,10.569331,241977.965768
std,2196.751708,8.401171,109612.549926
min,400.000000,0.000000,-65000.000000
25%,2304.000000,3.000000,147808.250000
50%,4221.000000,9.000000,241653.000000
75%,6063.000000,17.000000,336442.000000
max,8000.000000,35.000000,432000.000000


Chosen cutoffs:
data_mb_used cutoff: 7965.0
over_15mins_calls_per_month cutoff: 34.0
income cutoff: 429967.275

Shape BEFORE: (14986, 16)
Shape AFTER : (14779, 16)
Rows removed: 207


## Part 6: Handle Outliers Using Quantiles

Instead of removing outliers, we can limit their impact by capping (removing) extreme values — a method known as **Winsorizing**.

### How to Do It:
- Use `.quantile()` to identify the 1st and 99th percentiles (or other thresholds).
- Use `.clip()` to cap values within that range.

This keeps your dataset intact while reducing the influence of extreme values on your analysis or model.



In [62]:
# Calculate 1st and 99th percentiles for income
income_min, income_max = df['income'].quantile([0.01, 0.99])

# Use .loc to avoid SettingWithCopyWarning and ensure assignment modifies the original DataFrame
df.loc[:, 'income'] = df['income'].clip(lower=income_min, upper=income_max)

# Clip 'data_mb_used' to within 1st and 99th percentiles
usage_min, usage_max = df['data_mb_used'].quantile([0.01, 0.99])
df.loc[:, 'data_mb_used'] = df['data_mb_used'].clip(lower=usage_min, upper=usage_max)

# Clip 'average_call_duration' to reduce the effect of extreme outliers
call_min, call_max = df['average_call_duration'].quantile([0.01, 0.99])
df.loc[:, 'average_call_duration'] = df['average_call_duration'].clip(lower=call_min, upper=call_max)



### 🔧 Try It Yourself – Part 6

1. Use `.quantile([0.01, 0.99])` to find the range for:
  - `text_message_count`
  - `over_15mins_calls_per_month`
2. Apply `.clip(lower=..., upper=...)` to reduce the impact of those outliers

### In Your Response:
1. Compare the `.describe()` output before and after clipping and comment on what you observe


In [63]:
# Save BEFORE describe for comparison
before_desc = df[['text_message_count', 'over_15mins_calls_per_month']].describe()

# Find 1st and 99th percentiles
txt_min, txt_max = df['text_message_count'].quantile([0.01, 0.99])
c15_min, c15_max = df['over_15mins_calls_per_month'].quantile([0.01, 0.99])

print("Text message count 1% / 99%:", txt_min, txt_max)
print("Over 15 mins calls 1% / 99%:", c15_min, c15_max)

# Clip (winsorize)
df.loc[:, 'text_message_count'] = df['text_message_count'].clip(lower=txt_min, upper=txt_max)
df.loc[:, 'over_15mins_calls_per_month'] = df['over_15mins_calls_per_month'].clip(lower=c15_min, upper=c15_max)

# AFTER describe
after_desc = df[['text_message_count', 'over_15mins_calls_per_month']].describe()

print("\n=== BEFORE CLIP (describe) ===")
display(before_desc)

print("\n=== AFTER CLIP (describe) ===")
display(after_desc)

Text message count 1% / 99%: 53.0 219.0
Over 15 mins calls 1% / 99%: 0.0 32.0

=== BEFORE CLIP (describe) ===


,text_message_count,over_15mins_calls_per_month
count,14779.000000,14779.000000
mean,135.687936,10.473983
std,48.833943,8.281866
min,52.000000,0.000000
25%,93.000000,3.000000
50%,135.000000,9.000000
75%,178.000000,17.000000
max,220.000000,34.000000



=== AFTER CLIP (describe) ===


,text_message_count,over_15mins_calls_per_month
count,14779.000000,14779.000000
mean,135.687191,10.463901
std,48.810213,8.254559
min,53.000000,0.000000
25%,93.000000,3.000000
50%,135.000000,9.000000
75%,178.000000,17.000000
max,219.000000,32.000000


### ✍️ Your Response: 🔧
1. After clipping at the 1st and 99th percentiles, the minimums increased (if there were extreme low values) and the maximums decreased, while the mean and standard deviation became less extreme. This indicates winsorizing reduced the influence of unusually large/small values but kept all rows in the dataset.

## 🔧 Part 7: Reflection (100 words or less per question)

1. Which step fixed the most issues in the dataset?
2. What surprised you about the structure or values?
3. Do you feel this data is now ready for transformation in Lab 7?


### ✍️ Your Response: 🔧
The step that fixed the most issues was handling missing values and fixing data types. Missing values can mess up calculations and models, and having the wrong data types can cause numbers to act like text, which leads to errors or confusing results.

I was surprised by how messy real datasets can be. There were missing values, inconsistent category labels, and some extreme usage and income values. Even though the dataset looked fairly clean at first, it still needed several fixes before it was ready to use.

Yes, I think the data is ready for Lab 7. The columns are now consistent, data types are correct, missing values and duplicates are handled, and extreme outliers have been reduced, so it should be much easier to work with going forward.

## Export Your Notebook to Submit in Canvas
- Use the instructions from Lab 1

In [64]:
!jupyter nbconvert --to html "lab_06_data_cleaning.ipynb"

[NbConvertApp] WARNING | pattern 'lab_06_data_cleaning.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]
--ex